In [1]:
#4éme version
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime

def charger_donnees():
    # Chargement des fichiers CSV source
    df_vin = pd.read_csv("vinR.csv")
    print(f"vin.csv chargé: {len(df_vin)} entrées")
    
    df_producteur = pd.read_csv("producteurR.csv")
    print(f"producteur.csv chargé: {len(df_producteur)} entrées")
    
    df_recolte = pd.read_csv("recolteR.csv")
    print(f"recolte.csv chargé: {len(df_recolte)} entrées")
    
    return df_vin, df_producteur, df_recolte

def generer_donnees(df_vin, df_producteur, df_recolte, target_n=1_000_000):
    t0 = time.time()
    # Calcul du nombre de lignes à générer
    existantes = len(df_recolte)
    a_generer = max(0, target_n - existantes)
    print(f"{existantes} lignes existantes, {a_generer} à générer")
    
    if a_generer <= 0:
        return df_recolte.copy()
    
    # Index des combinaisons existantes
    seen = set(zip(df_recolte['Nprod'], df_recolte['Nvin'], df_recolte['Annee']))
    producteurs = df_producteur['Nprod'].unique()
    vins = df_vin['Nvin'].unique()
    nouvelles = []
    
    # Génération par batch pour performance
    while len(nouvelles) < a_generer:
        batch = min(5000, a_generer - len(nouvelles))
        np_nprod = np.random.choice(producteurs, batch)
        np_nvin = np.random.choice(vins, batch)
        np_annee = np.random.randint(1990, 2023, batch)
        np_qte = np.random.randint(50, 2001, batch)
        
        for i in range(batch):
            key = (int(np_nprod[i]), int(np_nvin[i]), int(np_annee[i]))
            if key not in seen:
                nouvelles.append({'Nprod': key[0], 'Nvin': key[1], 'Annee': key[2], 'Qte': int(np_qte[i])})
                seen.add(key)
                
                # Afficher progression par tranches de 25%
                if len(nouvelles) % (a_generer // 4) == 0:
                    print(f"Progression: {len(nouvelles)}/{a_generer}")
                
                if len(nouvelles) >= a_generer:
                    break
    
    # Concaténation et déduplication
    df_new = pd.DataFrame(nouvelles)
    df_full = pd.concat([df_recolte, df_new], ignore_index=True)
    
    # Suppression des doublons
    avant_dedup = len(df_full)
    df_full = df_full.drop_duplicates(subset=['Nprod','Nvin','Annee'], keep='first')
    apres_dedup = len(df_full)
    if avant_dedup > apres_dedup:
        print(f"Suppression de {avant_dedup - apres_dedup} doublons détectés")
    
    dt = time.time() - t0
    print(f"Génération terminée en {dt:.2f}s, total = {len(df_full)} lignes")
    return df_full

def sauvegarder(df):
    # Vérification supplémentaire pour la suppression des doublons
    avant_dedup = len(df)
    df = df.drop_duplicates(subset=['Nprod','Nvin','Annee'], keep='first')
    apres_dedup = len(df)
    if avant_dedup > apres_dedup:
        print(f"Suppression de {avant_dedup - apres_dedup} doublons supplémentaires avant sauvegarde")
    #to INT
    #df['Qte'] = df['Qte'].astype(int)
    
    # Format .dat pour SQL*Loader
    out_dat = 'recolte_augmente_1M.dat'
    df.to_csv(out_dat, sep='|', index=False, header=False)
    print(f"Fichier créé: {out_dat} ({os.path.getsize(out_dat)/(1024*1024):.2f} MB)")
    
    # Format .csv pour référence avec timestamp
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    out_csv = f'recolte_augmente_1M_{ts}.csv'
    df.to_csv(out_csv, index=False)
    print(f"Fichier créé: {out_csv} ({os.path.getsize(out_csv)/(1024*1024):.2f} MB)")
    
    # Vérification finale des doublons dans le fichier
    print("Vérification finale des doublons...")
    df_check = pd.read_csv(out_dat, sep='|', header=None, names=['Nprod', 'Nvin', 'Annee', 'Qte'])
    doublons = df_check.duplicated(subset=['Nprod', 'Nvin', 'Annee'], keep=False)
    if doublons.any():
        print(f"ATTENTION: {doublons.sum()} doublons trouvés dans le fichier final!")
        # Afficher un échantillon des doublons
        print("Exemple de doublons:")
        print(df_check[doublons].head(5))
    else:
        print("Pas de doublons détectés dans le fichier final.")

if __name__ == '__main__':
    print("=== Début génération données récolte ===")
    vin, prod, rec = charger_donnees()
    rec_aug = generer_donnees(vin, prod, rec, target_n=1_000_000)
    sauvegarder(rec_aug)
    print("=== Fin génération récolte ===")

=== Début génération données récolte ===
vin.csv chargé: 21089 entrées
producteur.csv chargé: 21089 entrées
recolte.csv chargé: 21089 entrées
21089 lignes existantes, 978911 à générer
Progression: 244727/978911
Progression: 489454/978911
Progression: 734181/978911
Progression: 978908/978911
Génération terminée en 2.04s, total = 1000000 lignes
Fichier créé: recolte_augmente_1M.dat (20.95 MB)
Fichier créé: recolte_augmente_1M_20250604_173431.csv (20.95 MB)
Vérification finale des doublons...
Pas de doublons détectés dans le fichier final.
=== Fin génération récolte ===
